# Shixuan Song




In [0]:
#load packages
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#load data
all_data = pd.read_csv("top150SplitCompanies.csv" ,index_col = 0)

In [0]:
all_data['class'] = np.where(all_data['rating']< 4, 0, 1)
y = all_data['class']
drop_columns = ['class', 'userId']
X = all_data.drop(labels= drop_columns, axis=1)

###KNN model

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import scale
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from sklearn.metrics import accuracy_score

#load data
all_data = pd.read_csv("top150SplitCompanies.csv" ,index_col = 0)
all_data = all_data.loc[:, 'budget':'Documentary']

def knn_model_acc(user):
    user_data = all_data.loc[all_data['userId'] == user] #test with user 414
    user_data['class'] = np.where(user_data['rating']< 4, 0, 1)
    y = user_data['class']
    drop_columns = ['class', 'userId']
    X = user_data.drop(labels= drop_columns, axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=223,
                                                    shuffle=True,
                                                    stratify = y)
    X_train = scale(X_train)
    X_test = scale(X_test)
    grid_params_knn = {
        'n_neighbors':[*range(2,15,1)],
        'weights':['uniform']
    }

    gs_knn = GridSearchCV(
         KNeighborsClassifier(),
         grid_params_knn,
         verbose = 1,
         cv = 10,
         refit=True,
         scoring='accuracy',
         n_jobs = -1)

    gs_knn_results = gs_knn.fit(X_train, y_train)
    knn_best_score = gs_knn.best_estimator_.score(X_test, y_test)*100
    #print('UserId:', userid, 'Accuracy:',knn_best_score)
    return knn_best_score

calculate KNN model accuracy

In [0]:
userId_list = all_data.userId.unique()
accuracy_list = []
for i in userId_list:
    accuracy_list.append(knn_model_acc(user = i))
    
print("In KNN model, the average accuracy for all users is ", round(np.mean(accuracy_list),2),"%")

###Gradient Boosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier 

def gb_model_acc(user):
    user_data = all_data.loc[all_data['userId'] == user] #test with user 414
    user_data['class'] = np.where(user_data['rating']< 4, 0, 1)
    y = user_data['class']
    drop_columns = ['class', 'userId']
    X = user_data.drop(labels= drop_columns, axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=223,
                                                    shuffle=True,
                                                    stratify = y)
    
    grid_params_gb = [{
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.5, 0.75, 1],
    "max_depth": [1, 2, 3, 4, 5, 10, 15, 20, None],
    #"criterion": ["gini",  "entropy"],
    "n_estimators":[5,10]
    }]

    gs_gb = GridSearchCV(
         GradientBoostingClassifier(),
         grid_params_gb,
         cv = 10,
         n_jobs = -1)

    gs_gb_results = gs_gb.fit(X_train, y_train)
    gb_best_score = gs_gb.best_estimator_.score(X_test, y_test)*100
    #print('UserId:', userid, 'Accuracy:',knn_best_score)
    return gb_best_score

Calculate average accuracy of KNN model

In [0]:
userId_list = all_data.userId.unique()
accuracy_list = []
for i in userId_list:
    accuracy_list.append(gb_model_acc(user = i))
    
print("In gradient boosting, the average accuracy for all users is ", round(np.mean(accuracy_list),2),"%")

NameError: ignored

# Tianchang Li


### Random Forest


In [0]:
################################### Random Forest Classifier ##################################
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

def RandomForest(X_train, y_train, X_test, y_test):
    
    ## Tuning hyperparameters
    param_grid = [{'max_depth': [5, 8, 10, 15, None],
                   'n_estimators': [120, 150, 180],
                   'criterion': ['gini','entropy']}]

    rd = GridSearchCV(estimator=RandomForestClassifier(random_state=223),
                      param_grid=param_grid,
                      n_jobs=-1,
                      scoring = 'accuracy',
                      refit=True,
                      cv=10)
    ## "refit" auto fit to the optimal model
    rd.fit(X_train, y_train)

    print('Best Accuracy: %.2f%%' % (rd.best_score_*100))
    print('Best Params:', rd.best_params_)
    print('Test Accuracy: %.2f%%' % (rd.best_estimator_.score(X_test, y_test)*100))

    # Feature selection for Random Forest
    imp = rd.best_estimator_.feature_importances_
    col_selected = topUser.drop(columns = ['rating','userId','vote_average','vote_count']).columns[imp > 0.001]
    X_selected = np.array(topUser[col_selected])

    X_train, X_test, y_train, y_test = train_test_split(X_selected, y,
                                                        test_size=0.3, 
                                                        random_state=223,
                                                        shuffle=True,
                                                        stratify = y)

    from sklearn.model_selection import StratifiedKFold

    cv = StratifiedKFold(n_splits=10, random_state=223, shuffle=True)
    kfold_acc = 0.

    for train_idx, valid_idx in cv.split(X_train, y_train):
        clf = RandomForestClassifier(criterion=rd.best_params_.get('criterion'),max_depth=rd.best_params_.get('max_depth'),n_estimators=rd.best_params_.get('n_estimators')).fit(X_train[train_idx], y_train[train_idx])
        y_pred = clf.predict(X_train[valid_idx])
        acc = np.mean(y_pred == y_train[valid_idx])*100
        kfold_acc += acc
    kfold_acc /= 10

    clf = RandomForestClassifier(criterion=rd.best_params_.get('criterion'),max_depth=rd.best_params_.get('max_depth'),n_estimators=rd.best_params_.get('n_estimators')).fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    test_acc = np.mean(y_pred == y_test)*100

    print('Kfold Accuracy: %.2f%%' % kfold_acc)
    print('Test Accuracy: %.2f%%' % test_acc)
    
    return np.mean(y_pred == y_test)
    
    
user150 = pd.read_csv("top150SplitCompanies.csv")
user150.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
# Loop through all users
userId_list = user150['userId'].unique()
accuracy_list = []
for elem in userId_list:
    topUser = user150[user150['userId'] == elem]
    X = np.array(topUser.drop(columns = ['rating','userId','vote_average','vote_count']))
    y = np.array(topUser['rating'])
    y[y < 4] = 0
    y[y >= 4] = 1
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=223,
                                                    shuffle=True,
                                                    stratify = y)
    accuracy_list.append(RandomForest(X_train, y_train, X_test, y_test))
    
mean_acc = np.mean(accuracy_list)  # Calculate average accuracy of all the individual user model
print("The average accurary of Random Forest is : %.2f%%" % (mean_acc*100))


### Decision Tree


In [0]:
############################### Decision Tree ###################################
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

def DecisionTree(X_train, y_train, X_test, y_test):
    
    param_grid = [{'max_depth': [3, 5, 7, 10, 13, None],'criterion': ['gini','entropy']}]
    dt = GridSearchCV(estimator=DecisionTreeClassifier(random_state=223),
                      param_grid=param_grid,
                      iid = True,
                      n_jobs=-1,
                      refit=True,
                      cv=10)
    ## "refit" auto fit to the optimal model
    dt.fit(X_train, y_train)

    print('Best Accuracy: %.2f%%' % (dt.best_score_*100))
    print('Best Params:', dt.best_params_)
    print('Test Accuracy: %.2f%%' % (dt.best_estimator_.score(X_test, y_test)*100))
#     PlotTree(X_train, y_train,dt)
    
    return dt.best_estimator_.score(X_test, y_test)
    
## Plot decision tree (Called at the end DecisionTree function)
def PlotTree(X_train,y_train,dt):
    from sklearn.tree import export_graphviz
    from sklearn.externals.six import StringIO  
    from IPython.display import Image  
    import pydotplus

    feature_cols = topUser.drop(columns = ['rating','userId','vote_average','vote_count']).columns
    clf = dt.best_estimator_
    clf.fit(X_train,y_train)

    dot_data = StringIO()
    export_graphviz(clf, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True,
                    feature_names = feature_cols,
                    class_names=['Not recommend','Recommend'])
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png('DT_max5.png')
    Image(graph.create_png())

    
user150 = pd.read_csv("top150SplitCompanies.csv")
user150.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
# Loop through all users
userId_list = user150['userId'].unique()
accuracy_list = []
for elem in userId_list:
    topUser = user150[user150['userId'] == elem]
    X = np.array(topUser.drop(columns = ['rating','userId','vote_average','vote_count']))
    y = np.array(topUser['rating'])
    y[y < 4] = 0
    y[y >= 4] = 1
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=223,
                                                    shuffle=True,
                                                    stratify = y)
    accuracy_list.append(DecisionTree(X_train, y_train, X_test, y_test))
    
mean_acc = np.mean(accuracy_list)  # Calculate average accuracy of all the individual user model
print("The average accurary of Decision tree is : %.2f%%" % (mean_acc*100))


### K-Mean Clustering

In [0]:
################################## K-Mean Clustering ##################################
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import numpy as np

# Loop through each user
def K_Mean (X_train, y_train):
    # Select the best "K"
    distortions = []
    for i in range(1, 15):
        km = KMeans(n_clusters=i, 
                    random_state=233)
        km.fit(X_train)
        distortions.append(km.inertia_)
    plt.plot(range(1, 15), distortions, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Distortion')
#     plt.savefig('distortion with K means',dpi = 300)
    
    ## Apply number of clusters as 4
    nclusters = 4
    kmeans = KMeans(n_clusters=nclusters, random_state=233)
    labels = kmeans.fit_predict(X_train)
    for i in range(nclusters):
        print('Mean of cluster_',i,':',np.mean(y_train[labels == i]))

    n_clus = len(np.unique(labels))
    n_recom = 0
    true_mean = []
    # Cluster level info
    for i in range(n_clus):
        true_mean.append(np.mean(y_train[labels == i]))
        print('Mean of cluster_',i,':',np.mean(y_train[labels == i]))
        if(np.mean(y_train[labels == i])>3.75): 
            n_recom += np.sum(labels == i)
            labels[labels == i] = 1   
        else:
            labels[labels == i] = 0

    y_label = y_train >= 4
    y_label = y_label.astype(int)
    acc = accuracy_score(y_label, labels)
    print('Accuracy score: %.2f%%' % (acc*100))
    print(confusion_matrix(y_label, labels))
    
    return (acc)



user150 = pd.read_csv("top150SplitCompanies.csv")
user150.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
# Loop through all users
userId_list = user150['userId'].unique()
accuracy_list = []
for elem in userId_list:
    topUser = user150[user150['userId'] == elem]
    X = np.array(topUser.drop(columns = ['rating','userId']))
    y = np.array(topUser['rating'])

    # Rescaling
    scaler = StandardScaler()
    X_trans = scaler.fit_transform(X)
    
    accuracy_list.append(K_Mean(X_trans, y))
    
mean_acc = np.mean(accuracy_list)  # Calculate average accuracy of all the individual user model
print("The average accurary of K-Mean clustering is : %.2f%%" % (mean_acc*100))


### Mean Shift Clustering

In [0]:
############################## Mean shift method ##############################
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def MeanShift(X_trans, y_train, X_test, y_test):
    
    bandwidth = estimate_bandwidth(X_trans, quantile=0.3)
    shift = MeanShift(bandwidth = bandwidth)
    shift.fit(X_trans)

    pred_label = shift.predict(X_test)
    pred_level = []
    for i in range(len(pred_label)):
        pred_level.append(true_mean[pred_label[i]])
    pred_level = np.array(pred_level)
    pred_level[pred_level < 3.75] = 0
    pred_level[pred_level >= 3.75] = 1
    y_label = y_test >= 4
    y_label = y_label.astype(int)
    # print('adjusted score:',adjusted_rand_score(y_label, pred_level))
    # print('Accuracy score: %.2f%%', accuracy_score(y_label, pred_level)*100)
    # fpr = np.sum(y_label[pred_level == 1] !=1)/np.sum(y_label[pred_level == 1] ==1)*100
    # fnr = np.sum(y_label[pred_level == 0] !=0)/np.sum(y_label[pred_level == 0] ==0)*100
    # print("false positive rate : %.2f%%" % fpr)
    # print("false negative rate : %.2f%%" % fnr)

    confmat = confusion_matrix(y_label, pred_level)
    fig, ax = plt.subplots(figsize=(2.5, 2.5))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.tight_layout()
    # plt.savefig('MeanShift_confmat.png', dpi=300)
    plt.show()

    return accuracy_score(y_label, pred_level)


# Split training, test set
from sklearn.model_selection import train_test_split

user150 = pd.read_csv("top150SplitCompanies.csv")
user150.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
# Loop through all users
userId_list = user150['userId'].unique()
accuracy_list = []
for elem in userId_list:
    topUser = user150[user150['userId'] == elem]
    X = np.array(topUser.drop(columns = ['rating','userId']))
    y = np.array(topUser['rating'])
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, 
                                                    random_state=233,
                                                    shuffle=True)
    
    # Rescaling
    scaler = StandardScaler()
    X_trans = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_train)
    
    accuracy_list.append(MeanShift(X_train, y_train, X_test, y_test))
    
mean_acc = np.mean(accuracy_list)  # Calculate average accuracy of all the individual user model
print("The average accurary of Mean Shift clustering is : %.2f%%" % (mean_acc*100))


### DBSCAN 

In [0]:
############################## DBSCAN ###########################
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def DBSCAN(X_trans, y_train):
    
    db = DBSCAN(eps=1, min_samples=2).fit(X_trans)
    labels = db.labels_
    n_clus = len(np.unique(labels))
    n_recom = 0
    true_mean = []
    # Cluster level info
    for i in range(-1,n_clus-1):
        if i == -1:
            true_mean.append(np.mean(y_train[labels == i]))
            print('Mean of cluster_',i,':',np.mean(y_train[labels == i]))
            labels[labels == i] = 0
        else:
            true_mean.append(np.mean(y_train[labels == i]))
            print('Mean of cluster_',i,':',np.mean(y_train[labels == i]))
            if(np.mean(y_train[labels == i])>3.75): 
                n_recom += np.sum(labels == i)
                labels[labels == i] = 1   
            else:
                labels[labels == i] = 0


    y_label = y_train >= 4
    y_label = y_label.astype(int)
    # print('adjusted score:',adjusted_rand_score(y_label, labels))
    acc = accuracy_score(y_label, labels)
    print('Accuracy score: %.2f%%' % (acc*100))
    # print("number of estimated clusters : %d" % n_clus)
    # print("number of recommended ones : %d" % n_recom)
    # fpr = np.sum(y_label[labels == 1] !=1)/np.sum(y_label[labels == 1] ==1)*100
    # fnr = np.sum(y_label[labels == 0] !=0)/np.sum(y_label[labels == 0] ==0)*100
    # print("false positive rate : %.2f%%" % fpr)
    # print("false negative rate : %.2f%%" % fnr)

    confmat = confusion_matrix(y_label, labels)
    fig, ax = plt.subplots(figsize=(2.5, 2.5))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

    plt.xlabel('Predicted label')
    plt.ylabel('True label')

    plt.tight_layout()
    # plt.savefig('DBSCAN_confmat.png', dpi=300)
    plt.show()
    
    return acc

    
user150 = pd.read_csv("top150SplitCompanies.csv")
user150.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
# Loop through all users
userId_list = user150['userId'].unique()
accuracy_list = []
for elem in userId_list:
    topUser = user150[user150['userId'] == elem]
    X = np.array(topUser.drop(columns = ['rating','userId']))
    y = np.array(topUser['rating'])

    # Rescaling
    scaler = StandardScaler()
    X_trans = scaler.fit_transform(X)
    
    accuracy_list.append(DBSCAN(X_trans, y))
    
mean_acc = np.mean(accuracy_list)  # Calculate average accuracy of all the individual user model
print("The average accurary of DBSCAN clustering is : %.2f%%" % (mean_acc*100))


# Maoze Wang

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [0]:
data = pd.read_csv('top150SplitCompanies.csv')  # Read dataset
#  Transform rating to binary 0 and 1
data['rating'][data['rating'] < 3.75] = 0  # Rating smaller than 3.75 will sorted to 'dislike(0)'
data['rating'][data['rating'] > 3.75] = 1  # Rating larger than 3.75 will sorted to 'like(1)'

In [0]:
# Split train and test dataset
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split


def my_train_test_split(userId_input):
    data_user = data.loc[data['userId'] == userId_input]  # Select User
    y = data_user[['rating']]  # Result
    X = data_user[['budget', 'original_language','popularity','year_from_now','revenue','runtime','status','vote_count',
              'Count_movies','Drama','Sci-Fi','Action','Crime','Thriller','Comedy','Horror','Adventure','Animation','Fantasy',
              'Musical', 'Romance','Mystery', 'IMAX','Western','War', 'Children','Documentary','Film-Noir', 'DE', 'US','JP','FR','GB',
              'AU','CN','IT','IE','CA','KR','CZ','BE','ES','RO','ZA','IN','IS','MX','HK','LU','TW','SG','MA','MT','JM','BS','DM','NZ',
              'CH','FJ', 'BR','AR','EC','CO','DK','RU','HU','NL','AE','EG','FI','AT','PK','PH','SE','IL','PL','CS','NO','MY','UA','AO',
              'CL','AF','TH','IR','MC','BG','PE','SK','GR','LT']]  # Variables
    X_train_sub, X_val, y_train_sub, y_val = train_test_split(X, y, test_size = 0.2, random_state = 123)  # Split data from same user with 0.2 test dataset.
    split_result = [X_train_sub, X_val, y_train_sub, y_val]  # A value that contain X train, test data and y train,test data.
    return split_result

1) Individual user models

In [0]:
# Create logistic lasso regression model

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
def logistic_lasso(X_train, y_train, X_test, y_test):
    log_l1 = LogisticRegression(penalty='l1', solver='liblinear')  # Create model with Logistic regression
    log_l1.fit(X_train,y_train)  # Fit the model with train data
    y_pred = log_l1.predict(X_test)  # Predict the test data
    acc = accuracy_score(y_test, y_pred)  # Calculate accuracy score
    return acc

In [0]:
userId_list = data['userId'].unique()
accuracy_list = []
for elem in userId_list:
    X_train = my_train_test_split(elem)[0]
    y_train = my_train_test_split(elem)[2]
    X_test = my_train_test_split(elem)[1]
    y_test = my_train_test_split(elem)[3]
    accuracy_list.append(logistic_lasso(X_train, y_train, X_test, y_test))
mean_acc = np.mean(accuracy_list)
print("In Lasso Regression model, the average accuracy for all users is ", round(mean_acc*100,2),"%")

2) Detail logistic lasso regression model for user 414

In [0]:
# Create a logistic regression for overall data for user 414 only
data_414 = data.loc[data['userId'] == 414]
y = data_414[['rating']]  # Result
X = data_414[['budget', 'original_language','popularity','year_from_now','revenue','runtime','status','vote_count',
          'Count_movies','Drama','Sci-Fi','Action','Crime','Thriller','Comedy','Horror','Adventure','Animation','Fantasy',
          'Musical', 'Romance','Mystery', 'IMAX','Western','War', 'Children','Documentary','Film-Noir', 'DE', 'US','JP','FR','GB',
          'AU','CN','IT','IE','CA','KR','CZ','BE','ES','RO','ZA','IN','IS','MX','HK','LU','TW','SG','MA','MT','JM','BS','DM','NZ',
          'CH','FJ', 'BR','AR','EC','CO','DK','RU','HU','NL','AE','EG','FI','AT','PK','PH','SE','IL','PL','CS','NO','MY','UA','AO',
          'CL','AF','TH','IR','MC','BG','PE','SK','GR','LT']]  # Variables
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X, y, test_size = 0.2, random_state = 123,stratify = y,shuffle = True)  # Split whole data into train test dataset

log_l1 = LogisticRegression(penalty='l1', solver='liblinear')  # Create model with Lasso regression
log_l1.fit(X_train_sub,y_train_sub)  # Fit the model with train data
y_pred = log_l1.predict(X_val)  # Predict the test data
acc = accuracy_score(y_val, y_pred)  # Calculate accuracy score
print("In Lasso Regression model, the accuracy for user 414 is", round(acc*100,2),"%")

In [0]:
print(log_l1.coef_)  # Print the coefficients of model

In [0]:
# Convert coefficients to change in probability
import math
print('budget: ' + str(math.exp(-1.70989788e-08)/(1 + math.exp(-1.70989788e-08))))  # budget
print('original_language: ' + str(math.exp(4.97632901e-02)/(1 + math.exp(4.97632901e-02))))  # original_language
print('popularity: ' + str(math.exp(1.55152778e-02)/(1 + math.exp(1.55152778e-02))))  # popularity
print('year_from_now: ' + str(math.exp(1.96388026e-02)/(1 + math.exp(1.96388026e-02))))  # year_from_now
print('revenue: ' + str(math.exp(-6.49735709e-10)/(1 + math.exp(-6.49735709e-10))))  # revenue
print('runtime: ' + str(math.exp(1.42996047e-02)/(1 + math.exp( 1.42996047e-02))))  # runtime
print('vote_count: ' + str(math.exp(4.98632801e-04)/(1 + math.exp(4.98632801e-04))))  # vote_count
print('Count_movies: ' + str(math.exp( -2.03109209e-03)/(1 + math.exp( -2.03109209e-03))))  # Count_movies

In [0]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
confmat = confusion_matrix(y_val, y_pred)  # Create confusion matrix
confmat

fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.8)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

# Print accuracy, precision, and recall
print("Accuracy:",accuracy_score(y_val, y_pred))
print("Precision:",precision_score(y_val, y_pred))
print("Recall:",recall_score(y_val, y_pred))

In [0]:
# Plot ROC Curve
from sklearn.metrics import confusion_matrix
from sklearn import metrics
y_pred_proba = log_l1.predict_proba(X_val)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_val,  y_pred_proba)
auc = metrics.roc_auc_score(y_val, y_pred_proba)
plt.plot(fpr,tpr,label="logistic lasso, auc="+str(auc))
plt.plot([0, 1],[0, 1],linestyle='--',color=(0.6, 0.6, 0.6),label='Random guessing')
plt.legend(loc=4)
plt.show()

# Xuchen Xue


###Build tree regression model

In [0]:
import numpy as np   
import matplotlib.pyplot as plt   
import pandas as pd  
user150 = pd.read_csv("top150SplitCompanies.csv")

FileNotFoundError: ignored

In [0]:
############################### Decision Tree Regression ###################################
# Split training, test set

def tree_regression(user):
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.metrics import accuracy_score
    topUser = user150[user150['userId'] == user]
    X = np.array(topUser.drop(columns = ['rating','userId','vote_average','vote_count']))
    y = np.array(topUser['rating'])
    y[y < 4] = 0
    y[y >= 4] = 1

    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=223,
                                                    shuffle=True,
                                                    stratify = y)

    dt_regression = DecisionTreeRegressor(random_state = 123,criterion='mse',max_depth=3) 
    dt_regression.fit(X_train, y_train)
    y_pred = np.array(dt_regression.predict(X_test))
    for i in range(len(y_pred)):
        if y_pred[i] < np.mean(y_pred):
            y_pred[i] = 0
        else:
            y_pred[i] = 1

    num_correct_predictions = (y_pred== y_test).sum()
    accuracy = (num_correct_predictions / y_test.shape[0]) * 100

    return accuracy

### Calculate average accuracy of all the individual user model

In [0]:
userId_list = user150['userId'].unique()
accuracy_list = []
for i in userId_list:
    accuracy_list.append(tree_regression(user = i))
    
print("In decision tree regression model, the average accuracy for all users is ", round(np.mean(accuracy_list),2),"%")

### Plot Decision Tree regression 

In [0]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

feature_cols = topUser.drop(columns = ['rating','userId','vote_average','vote_count']).columns
clf = DecisionTreeRegressor(random_state = 123,criterion='mse',max_depth=3) 
clf.fit(X_train,y_train)
        
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,
                feature_names = feature_cols)
pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('tree_regression.png')
Image(graph.create_png())
## value close to 1 means recommend (likely to have a rating >= 4); 
## value close to 0 means don't recommend (likely to have a rating <4)

# Final Model - KNN 

In [0]:
#all_data = all_data.loc[all_data['userId'] == userid]
all_data['class'] = np.where(all_data['rating']< 4, 0, 1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=223,
                                                    shuffle=True,
                                                    stratify = y)



In [0]:
y = all_data['class']
drop_columns = ['class', 'userId']
X = all_data.drop(labels= drop_columns, axis=1)

X_train = X_train.loc[:, 'budget':'Documentary']
X_train = scale(X_train)
X_test = X_test.loc[:, 'budget':'Documentary']
X_test = scale(X_test)

grid_params_knn = {
        'n_neighbors':[*range(2,15,1)],
        'weights':['uniform']
    }

gs_knn = GridSearchCV(
         KNeighborsClassifier(),
         grid_params_knn,
         verbose = 1,
         cv = 10,
         refit=True,
         scoring='accuracy',
         n_jobs = -1)

gs_knn_results = gs_knn.fit(X_train, y_train)
knn_best_score = gs_knn.best_estimator_.score(X_test, y_test)*100
#y_knn_predictions = gs_knn.best_estimator_.predict(X_test)
print(knn_best_score, '%')

In [0]:
print('The best hyperparameter for KNN Model:',gs_knn.best_params_)